In [1]:
import requests
import json
import pandas as pd
import numpy as np
import requests
import plotly.graph_objects as go
from datetime import datetime as dt
from bs4 import BeautifulSoup
from math import log, log2, ceil
import re
import os
import sys
import os
import pickle
from sqlalchemy import Table, Column, Integer, String, MetaData, create_engine
import sqlite3, psycopg2
from unidecode import unidecode

In [2]:
def save_object(obj, filename):
    with open(filename, 'wb') as outp:
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
    
    
def load_object(filename):
    with open(filename, "rb") as file:
        return pickle.load(file)
    

def get_ingredient_list(link, html):
    ingredient_list = html.find("ul", attrs={"class": "ingredient-list"})

    ret = list()
    if ingredient_list is None:
        ingredients = [li for li in html.find(
            "ul", attrs={"class": "structured-ingredients__list"}).find_all("li")]
        for el in ingredients:
            ret.append({
                "name": el.find("p").find("span", attrs={"data-ingredient-name": "true"}).text if el.find("p").find("span", attrs={"data-ingredient-name": "true"}) is not None else None,
                "unit": el.find("p").find("span", attrs={"data-ingredient-unit": "true"}).text if el.find("p").find("span", attrs={"data-ingredient-unit": "true"}) is not None else None,
                "quantity": el.find("p").find("span", attrs={"data-ingredient-quantity": "true"}).text if el.find("p").find("span", attrs={"data-ingredient-quantity": "true"}) is not None else None
            })
    else:
        ingredients = [
            re.sub("\n", "", li.text) for li in html.find("ul", attrs={"class": "ingredient-list"}).find_all("li")
        ]
        for el in ingredients:
            if re.search("^garnish", el, flags=re.IGNORECASE):
                quantity = None
                unit = "garnish"
                name = re.sub("^Garnish: ", "", el).strip()
            else:
                quantity = re.search("^[0-9/ ]+", el).group(0).strip()
                unit = re.sub(quantity, "", re.search(
                    "^[0-9/ ]+ [A-Za-z]+ ", el).group(0).strip()).strip()
                name = re.sub(f"{quantity}|{unit}", "", el).strip()
            ret.append({
                "name": name,
                "unit": unit,
                "quantity": quantity
            })
    image = html.find(
        "figure",
        attrs={"class": "comp figure figure--primary-image"}
    ).find(
        "div",
        attrs={"class", "img-placeholder"}
    ).find("img")["src"]

    return {
        "recipe_name": re.sub(" Cocktail Recipe$", "", html.find("title").text),
        "image": image,
        "link": link,
        "ingredients": ret
    }


def get_recipe_ingredients(recipe_links, recipes):
    for link in recipe_links:
        text = requests.get(link).text
        html = BeautifulSoup(text, 'html.parser')
            
        try:
            recipe = get_ingredient_list(link, html)
            recipes.append(recipe)

        except AttributeError as e:
            try:
                sub_links = [a["href"] for a in html.find_all(
                    "a", attrs={"class": "mntl-sc-block-heading__link"})]
                get_recipe_ingredients(sub_links, recipes)
            except AttributeError as e:
                print(f"{link} - No recipe or sub-links")

    return recipes

In [ ]:
liquor_links = {
    "bourbon": "https://www.liquor.com/bourbon-cocktails-4779435",
    "vodka": "https://www.liquor.com/vodka-cocktails-4779437",
    "rum": "https://www.liquor.com/rum-cocktails-4779434",
    "scotch": "https://www.liquor.com/scotch-cocktails-4779431",
    "rye_whiskey": "https://www.liquor.com/rye-whiskey-cocktails-4779433",
    "other_whiskey": "https://www.liquor.com/whiskey-cocktails-4779430",
    "tequila_mezcal": "https://www.liquor.com/tequila-and-mezcal-cocktails-4779429",
    "cognac_brandy": "https://www.liquor.com/brandy-cocktails-4779428",
    "other": "https://www.liquor.com/other-cocktails-4779427",
    "gins": "https://www.liquor.com/gin-cocktails-4779436"
}

In [ ]:
recipes = list()
all_recipes = list()
for liquor, liquor_link in liquor_links.items():
    print(liquor)
    text = requests.get(liquor_link).text
    html = BeautifulSoup(text, 'html.parser')
    recipe_links = [
        a["href"] for a in html.find_all(
            "a", 
            attrs={"class": "comp mntl-card-list-items mntl-document-card mntl-card card"}
        )
    ]
    
    all_recipes.append({"liquor": liquor, "recipes": get_recipe_ingredients(recipe_links, recipes)})

In [ ]:
master_list = list()
for liquor in all_recipes:
    for recipe in liquor.get("recipes"):
        if recipe not in master_list:
            master_list.append(recipe)

In [ ]:
len(master_list)

In [ ]:
pd.DataFrame(master_list).to_parquet("cocktail.parquet")

In [ ]:
save_object(master_list, "cocktails.pkl")

In [ ]:
cocktails = load_object("cocktails.pkl")

In [ ]:
cocktails_df = pd.DataFrame(cocktails)

In [ ]:
cocktails_df = cocktails_df.reset_index().rename(columns={"index":"cocktail_id"})

In [ ]:
cocktails_df

In [ ]:
cocktails_exploded = cocktails_df.explode("ingredients")

In [ ]:
cocktails_exploded.reset_index()

In [ ]:
cocktails_ingredients = pd.concat(
    [cocktails_exploded.reset_index(), pd.json_normalize(cocktails_exploded['ingredients'])], 
    axis=1
).drop(["index", "ingredients"], axis=1).rename(columns={"name": "ingredient"})

In [ ]:
list(os.environ.keys())

In [3]:
DB_HOST = os.environ['COCKTAILS_HOST']
DB_PW = os.environ["COCKTAILS_PWD"]
DB_PORT = os.environ["COCKTAILS_PORT"]
DB_USER = os.environ["COCKTAILS_USER"]
DB_NAME = os.environ["COCKTAILS_DB"]
# DATABASE_URL = "/Users/jeremycolon/jc_git/cocktails-database/data/db.sqlite"

In [6]:
try:
    # link to your database
    conn = psycopg2.connect(
        database=DB_NAME,
        user=DB_USER,
        password=DB_PW,
        host=DB_HOST,
        port=DB_PORT,
        sslmode="require"
    )
except Exception as e:
    print("I am unable to connect to the database", e)

In [7]:
conn.close()

In [ ]:
cocktails_ingredients.drop(["image","link","recipe_name"], axis=1).to_sql(
    "cocktails_ingredients", con=conn, if_exists="replace"
)

In [ ]:
cocktails_df.drop(["ingredients"], axis=1).to_sql(
    "cocktails", con=conn, if_exists="replace", index=False
)

In [ ]:
ingredient_map = pd.read_excel("../data/ingredient_map.xlsx", sheet_name="Sheet2")

In [ ]:
ingredient_map.to_sql(
    "ingredient_map", con=conn, if_exists="replace", index=False
)

In [5]:
res = conn.execute(
        """
        SELECT ci.ingredient, count(distinct(c.cocktail_id))
        FROM cocktails c
        JOIN cocktails_ingredients ci
        ON c.cocktail_id = ci.cocktail_id
        where ci.ingredient = '1'
        group by 1
        order by 2 desc
        """
    ).fetchall()
res

AttributeError: 'psycopg2.extensions.connection' object has no attribute 'execute'

In [ ]:
res = conn.execute(
        """
        SELECT ci.ingredient, c.cocktail_id, c.link
        FROM cocktails c
        JOIN cocktails_ingredients ci
        ON c.cocktail_id = ci.cocktail_id
        where ci.ingredient = '8 to 10'
        """
    ).fetchall()
res

In [ ]:
ingredients = set(
    [
        re.sub("[*]|,$","",unidecode(ig.get("name").lower()).title().strip()) if ig.get("name") is not None else None 
        for cocktail in cocktails 
        for ig in cocktail.get("ingredients")
    ]
)

In [ ]:
garnishes = set(
    [
        re.sub("[*]|,$","",unidecode(ig.get("name").lower()).title().strip()) if re.search("^Garnish: ", str(ig.get("name")), flags=re.IGNORECASE) or ig.get("unit") == "garnish" else None 
        for cocktail in cocktails 
        for ig in cocktail.get("ingredients")
    ]
)

In [ ]:
bitters = set(
    [
        re.sub("[*]|,$","",unidecode(ig.get("name").lower()).title().strip()) if re.search("bitter", str(ig.get("name"))) else None 
        for cocktail in cocktails 
        for ig in cocktail.get("ingredients")
    ]
)

In [ ]:
syrups = set(
    [
        re.sub("[*]|,$","",unidecode(ig.get("name").lower()).title().strip()) if re.search("syrup", str(ig.get("name"))) else None 
        for cocktail in cocktails 
        for ig in cocktail.get("ingredients")
    ]
)

In [ ]:
ingredients = ingredients - garnishes - bitters - syrups
# ingredients = ingredients - bitters
# ingredients = ingredients - syrups

In [ ]:
[i for i in ingredients if re.search("\\bgin\\b", i, flags=re.IGNORECASE)]

In [ ]:
pd.set_option("display.max_rows", 100)

In [ ]:
alc = "vodka"
res = conn.execute(
        f"""
        with alcohol as (
            select distinct cocktail_id
            from cocktails_ingredients ci
            LEFT JOIN ingredient_map im
            ON ci.ingredient = im.ingredient
            -- where alcohol_type = '{alc}'
        ), ingredients as (
            SELECT ci.*, 
            COALESCE(im.mapped_ingredient, ci.ingredient) as mapped_ingredient, 
            im.alcohol_type
            FROM cocktails_ingredients  ci
            LEFT JOIN ingredient_map im
            ON ci.ingredient = im.ingredient
        )
        select i.*
        from ingredients i
        join alcohol a
        on i.cocktail_id = a.cocktail_id
        where i.alcohol_type != '{alc}' or i.alcohol_type is null;
        """
    ).fetchall()

In [ ]:
results = pd.DataFrame(
    res, 
    columns=["index", "cocktail_id", "ingredient", "unit", "quantity", "mapped_ingredient", "alcohol_type"]
)

In [ ]:
results

In [ ]:
results.groupby("mapped_ingredient").agg({
    "cocktail_id": "nunique",
    "index": "count"
}).reset_index().assign(
    total=len(results["cocktail_id"].unique()),
    perc_of_total=lambda row: row["cocktail_id"]/row["total"]
).sort_values("cocktail_id", ascending=False).head(100)